# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import re
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [18]:
# load data from database
engine=create_engine('sqlite:///MessagesDataframe.db')
df = pd.read_sql_table("MessagesDataframe",engine)
df.head()

,genre,id,message,original,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,direct,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,direct,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,direct,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,direct,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,direct,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:

(df.drop(['genre','id','message','original'],axis=1).sum(axis=0)/df.shape[0])

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

In [5]:
df.shape

(26216, 40)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """Tokenize and lemmatize text data"""
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Pipeline with vectorizer, TFIDF and MultiOutputClassifier
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [4]:
X = df['message'] # Leave original message apart to avoid multi correlation
y = df.drop(['id','message','original','genre'],axis=1) # Drop non categories columns


In [9]:
print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


In [5]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [122]:
# Fit the training data
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x128df88b0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [128]:
y_pred = pipeline.predict(X_test)

'related'

In [143]:
# Iterate over columns
y_pred = pipeline.predict(X_test)
for i in range(len(y.columns)):
    print(f'Scores for column {y.columns[i]} is:')
    print(f'{classification_report(y_pred[:,i],y_test.iloc[:,i])}')

Scores for column related is:
              precision    recall  f1-score   support

           0       0.26      0.72      0.38       667
           1       0.97      0.80      0.88      7191
           2       0.07      0.57      0.12         7

    accuracy                           0.79      7865
   macro avg       0.43      0.70      0.46      7865
weighted avg       0.91      0.79      0.83      7865

Scores for column request is:
              precision    recall  f1-score   support

           0       0.99      0.89      0.94      7227
           1       0.43      0.89      0.58       638

    accuracy                           0.89      7865
   macro avg       0.71      0.89      0.76      7865
weighted avg       0.94      0.89      0.91      7865

Scores for column offer is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865
           1       0.00      0.00      0.00         0

    accuracy                           1

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      7283
           1       0.73      0.88      0.80       582

    accuracy                           0.97      7865
   macro avg       0.86      0.93      0.89      7865
weighted avg       0.97      0.97      0.97      7865

Scores for column cold is:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7844
           1       0.09      0.76      0.17        21

    accuracy                           0.98      7865
   macro avg       0.55      0.87      0.58      7865
weighted avg       1.00      0.98      0.99      7865

Scores for column other_weather is:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      7863
           1       0.00      1.00      0.01         2

    accuracy                           0.95      7865
   macro avg       0.50      0.97      0.49      7865
weighted av

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
from skopt import BayesSearchCV
import optuna

In [13]:
# Define parameters and create gridsearch
parameters = parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
        'clf__estimator__min_samples_leaf': [1, 2, 3],
        
    }

In [14]:
# Create a bayesian search with parameters
searchcv = BayesSearchCV(
    pipeline,
    search_spaces=parameters,
    n_iter=10,
    cv=3
)

In [17]:
searchcv.fit(X_train,y_train)

BayesSearchCV(cv=3,
              estimator=Pipeline(steps=[('vect',
                                         CountVectorizer(tokenizer=<function tokenize at 0x129e383a0>)),
                                        ('tfidf', TfidfTransformer()),
                                        ('clf',
                                         MultiOutputClassifier(estimator=RandomForestClassifier()))]),
              n_iter=10,
              search_spaces={'clf__estimator__min_samples_leaf': [1, 2, 3],
                             'clf__estimator__min_samples_split': [2, 3, 4],
                             'clf__estimator__n_estimators': [50, 100, 200]})

In [18]:
searchcv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x129e383a0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=4,
                                                                        n_estimators=200)))])

In [19]:
searchcv.best_params_

OrderedDict([('clf__estimator__min_samples_leaf', 1),
             ('clf__estimator__min_samples_split', 4),
             ('clf__estimator__n_estimators', 200)])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [6]:
opt_pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_leaf=1,min_samples_split=4,n_estimators=200)))
    ])


In [8]:
# Fit the optimum pipeline
opt_pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x1054f19d0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=4,
                                                                        n_estimators=200)))])

In [19]:
type(y_test.columns.to_list())

list

In [22]:
# Test the predictions

opt_preds= opt_pipeline.predict(X_test)


In [69]:
# Import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score


In [29]:
# Loop and store metrics
accuracy = []
precision = []
recall = []
for i in range(len(y.columns)):
    accuracy.append(accuracy_score(opt_preds[:,i],y_test.iloc[:,i]))
    precision.append(precision_score(opt_preds[:,i],y_test.iloc[:,i],average='macro',zero_division=0))
    recall.append(recall_score(opt_preds[:,i],y_test.iloc[:,i],average='macro',zero_division=0))

In [42]:
metrics_df = pd.DataFrame([accuracy,precision,recall],columns=y_test.columns,index=['accuracy','precision','recall'])
metrics_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
accuracy,0.790718,0.896503,0.995423,0.775842,0.920025,0.948252,0.975842,0.981437,0.969358,1.0,...,0.986904,0.956643,0.868404,0.950286,0.934011,0.989320,0.969358,0.979402,0.947616,0.858360
precision,0.422319,0.713271,0.500000,0.753589,0.519915,0.518803,0.507747,0.499870,0.518039,1.0,...,0.500000,0.500000,0.792026,0.708935,0.701821,0.499936,0.872034,0.532034,0.504752,0.661764
recall,0.706894,0.899086,0.497711,0.776728,0.760934,0.855328,0.862979,0.490843,0.806399,1.0,...,0.493452,0.478322,0.865476,0.923504,0.859759,0.494723,0.931147,0.912888,0.873855,0.854608


In [45]:
metrics_df['mean_score'] =[metrics_df.iloc[0].mean(),metrics_df.iloc[1].mean(),metrics_df.iloc[2].mean()]
metrics_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,mean_score
accuracy,0.790718,0.896503,0.995423,0.775842,0.920025,0.948252,0.975842,0.981437,0.969358,1.0,...,0.956643,0.868404,0.950286,0.934011,0.989320,0.969358,0.979402,0.947616,0.858360,0.945815
precision,0.422319,0.713271,0.500000,0.753589,0.519915,0.518803,0.507747,0.499870,0.518039,1.0,...,0.500000,0.792026,0.708935,0.701821,0.499936,0.872034,0.532034,0.504752,0.661764,0.578396
recall,0.706894,0.899086,0.497711,0.776728,0.760934,0.855328,0.862979,0.490843,0.806399,1.0,...,0.478322,0.865476,0.923504,0.859759,0.494723,0.931147,0.912888,0.873855,0.854608,0.766021


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
import xgboost as xgb

In [10]:
# Create pipeline only for text
alternative_pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf',MultiOutputClassifier(xgb.XGBClassifier(objective="binary:logistic")))
    ])


In [61]:
# Transform to later use pycaret
alternative_pipeline.fit(X_train,y_train)

[12:09:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:09:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:10:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:10:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objectiv

[12:16:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:17:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:17:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:17:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x1054f19d0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None,
                                                               scale_pos_weight=None,
                                                               subsample=None,
                                                               tree_method=None,
                                                               validate_parameters=None,
                                                               verbosity=None)))])

In [63]:
alt_preds= alternative_pipeline.predict(X_test)
alt_accuracy = []
alt_precision = []
alt_recall = []
for i in range(len(y.columns)):
    alt_accuracy.append(accuracy_score(alt_preds[:,i],y_test.iloc[:,i]))
    alt_precision.append(precision_score(alt_preds[:,i],y_test.iloc[:,i],average='macro',zero_division=0))
    alt_recall.append(recall_score(alt_preds[:,i],y_test.iloc[:,i],average='macro',zero_division=0))

In [64]:
metrics_alt_df = pd.DataFrame([alt_accuracy,alt_precision,alt_recall],columns=y_test.columns,index=['accuracy','precision','recall'])
metrics_alt_df['mean_score'] =[metrics_alt_df.iloc[0].mean(),metrics_alt_df.iloc[1].mean(),metrics_alt_df.iloc[2].mean()]
metrics_alt_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,mean_score
accuracy,0.812079,0.902861,0.995296,0.767578,0.926891,0.957533,0.977877,0.981691,0.971138,1.0,0.965544,0.955245,0.947743,0.989955,0.980420,0.989955,0.968976,0.970121,0.874253,0.935283,0.960331,0.962111,0.980801,0.993643,0.989447,0.996186,0.986777,0.956008,0.882645,0.957152,0.943929,0.989955,0.971519,0.984107,0.947362,0.868023,0.951123
precision,0.513977,0.770292,0.499936,0.747978,0.634400,0.656930,0.582421,0.506815,0.637470,1.0,0.816618,0.870746,0.792270,0.733621,0.647726,0.586764,0.619921,0.746533,0.582924,0.532926,0.616385,0.685163,0.623376,0.499872,0.531409,0.499872,0.514305,0.522065,0.824946,0.778681,0.795359,0.577735,0.889282,0.694542,0.559228,0.721617,0.661503
recall,0.762201,0.858042,0.497711,0.764597,0.772183,0.845643,0.837595,0.740968,0.776138,1.0,0.875408,0.897129,0.863040,0.871086,0.788863,0.916209,0.773074,0.877370,0.726200,0.706736,0.834636,0.850416,0.805354,0.496948,0.662017,0.498220,0.707923,0.695457,0.870896,0.902105,0.852621,0.790992,0.929454,0.873995,0.729044,0.821120,0.790872


<b> XGB has a better performance in 3 metrics (specially precision) than RandomForestClassifier

### 9. Export your model as a pickle file

In [11]:
import pickle
import joblib

In [7]:
with open('mypickle.pickle', 'wb') as f:
    pickle.dump(alternative_pipeline, f)

NameError: name 'alternative_pipeline' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.